In [75]:
import requests
from bs4 import BeautifulSoup
from PIL import Image, ImageDraw, ImageFont
import copy
import textwrap
from pathlib import WindowsPath
from datetime import date
import os


In [162]:
today = date.today()
# today = '2024-03-18'

In [77]:
# content_dic['DRUGIE CZYTANIE'][4] = 'Łaska wam i pokój od Boga ojca naszego i Pana Jezusa Chrystusa! Bogu mojemu dziękuję wciąż za was, za łaskę daną wam w Chrystusie Jezusie. W Nim to bowiem zostaliście wzbogaceni we wszystko: we wszelkie słowo i wszelkie poznanie, bo świadectwo Chrystusowe utrwaliło się w was. Nie doznajecie tedy braku żadnej łaski, oczekując objawienia się Pana naszego, Jezusa Chrystusa. On też będzie umacniał was aż do końca, abyście byli bez zarzutu w dzień Pana naszego, Jezusa Chrystusa.'
# content_dic['EWANGELIA'][3] = 'Jezus powiedział do swoich uczniów:'
# content_dic['EWANGELIA'].insert(4,'«Uważajcie, czuwajcie, bo nie wiecie, kiedy czas ten nadejdzie. Bo rzecz ma się podobnie jak z człowiekiem, który udał się w podróż. Zostawił swój dom, powierzył swym sługom staranie o wszystko, każdemu wyznaczył zajęcie, a odźwiernemu przykazał, żeby czuwał.')
# content_dic['EWANGELIA']

In [103]:
URL = f"https://liturgia.wiara.pl/kalendarz/67b53.Czytania-mszalne/{str(today)}"


page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

content_list = soup.find_all("div", "txt__rich-area")[1].get_text().split('\n')

if content_list[-1] == '':
    del content_list[-1]

In [127]:
content_list

['PIERWSZE CZYTANIE',
 'Dn 13, 41c-62',
 'Czytanie z Księgi proroka Daniela',
 'Na wygnaniu w Babilonie zgromadzenie Izraela skazało na śmierć Zuzannę, fałszywie oskarżoną przez dwóch starców.',
 'Wtedy Zuzanna zawołała donośnym głosem: «Wiekuisty Boże, który znasz to, co jest ukryte, i wiesz wszystko, zanim się stanie. Ty wiesz, że złożyli fałszywe oskarżenie przeciw mnie. Oto umieram, chociaż nie uczyniłam nic z tego, o co mnie oni złośliwie obwiniają».',
 'A Pan wysłuchał jej głosu. Gdy ją prowadzono na stracenie, wzbudził Bóg świętego ducha w młodzieńcu imieniem Daniel. Zawołał on donośnym głosem: «Jestem czysty od jej krwi!» Cały zaś lud zwrócił się do niego, mówiąc: «Co oznacza to słowo, które wypowiedziałeś?» On zaś, powstawszy wśród nich, powiedział: «Czy tak bardzo jesteście nierozumni, Izraelici, że skazujecie córkę izraelską bez dochodzenia i pewności? Wróćcie do sądu, bo ci ją fałszywie obwinili».',
 'Cały lud zawrócił w pośpiechu. Starsi zaś powiedzieli: «Usiądź tu wśród n

In [119]:
# content_list = content_list[content_list.index('PIERWSZE CZYTANIE KRÓTSZE'):]
# content_list[0] = content_list[0].replace('PIERWSZE CZYTANIE KRÓTSZE', 'PIERWSZE CZYTANIE')
# content_list
try:
    i = content_list.index('EWANGELIA ')
    content_list[i] = content_list[i].replace('EWANGELIA ', 'EWANGELIA')
except:
    pass

In [81]:
# content_list.index('albo:')

In [82]:
URL

'https://liturgia.wiara.pl/kalendarz/67b53.Czytania-mszalne/2024-03-18'

In [83]:
# content_list[content_list.index('EWANGELIA')]
# content_dic['PIERWSZE CZYTANIE'] = content_dic['PIERWSZE CZYTANIE'][:-5]

# content_dic['PSALM RESPONSORYJNY'].insert(11, 'Refren: Po całej ziemi ich głos się rozchodzi.')
# content_dic['PSALM RESPONSORYJNY']

In [128]:
if 'EWANGELIA DŁUŻSZA' in content_list:
    cut_points = ['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'DRUGIE CZYTANIE', 'EWANGELIA DŁUŻSZA', 'EWANGELIA KRÓTSZA']
elif 'DRUGIE CZYTANIE' in content_list:
    cut_points = ['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'DRUGIE CZYTANIE', 'EWANGELIA']
else:
    cut_points = ['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA']

content_dic = {}
staging_list = [] 


for text in content_list:
    if 'Liturgia Słowa' in text or '' == text:
        continue

    # content_list[content_list.index(text)] = text.strip()
    # text = text.strip()
    
    if text in cut_points:

        for text in content_list[content_list.index(text)+1:]:
            if '' == text:
                continue
            if text == 'albo':
                content_dic[cut_points[0]] = staging_list
                staging_list = [] 
                del cut_points[0]
                break
            if text not in cut_points:
                staging_list.append(text)
                
            else:
                content_dic[cut_points[0]] = staging_list
                staging_list = [] 
                del cut_points[0]
                break
        
        content_dic[cut_points[0]] = staging_list

content_dic 

        

IndexError: list index out of range

In [129]:
content_dic.keys()


dict_keys(['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA'])

In [130]:
if 'EWANGELIA KRÓTSZA' in content_list:
    content_dic['EWANGELIA KRÓTSZA'].insert(1, content_dic['EWANGELIA DŁUŻSZA'][1])
    content_dic['EWANGELIA KRÓTSZA']

In [131]:
from functools import reduce
count = reduce(lambda x, y: x + 1 if 'Refren' in y else x, content_dic['PSALM RESPONSORYJNY'], 0)
if len(content_dic['PSALM RESPONSORYJNY']) > 6 and count == 1:
    content_dic['PSALM RESPONSORYJNY'].insert(6, 'Refren')
    content_dic['PSALM RESPONSORYJNY'].insert(11, 'Refren')
    content_dic['PSALM RESPONSORYJNY'].insert(16, 'Refren')
content_dic['PSALM RESPONSORYJNY']

['Ps 23 (22), 1b-3a. 3b-4. 5. 6 (R.: por. 4ab)',
 'Refren: Nic mnie nie trwoży, bo Ty jesteś ze mną.',
 'Pan jest moim pasterzem, †',
 'niczego mi nie braknie, *',
 'pozwala mi leżeć na zielonych pastwiskach.',
 'Prowadzi mnie nad wody, gdzie mogę odpocząć, *',
 'orzeźwia moją duszę.',
 'Refren.',
 'Wiedzie mnie po właściwych ścieżkach *',
 'przez wzgląd na swoją chwałę.',
 'Chociażbym przechodził przez ciemną dolinę, †',
 'zła się nie ulęknę, bo Ty jesteś ze mną. *',
 'Kij Twój i laska pasterska są moją pociechą.',
 'Refren.',
 'Stół dla mnie zastawiasz *',
 'na oczach mych wrogów.',
 'Namaszczasz mi głowę olejkiem, *',
 'kielich mój pełny po brzegi.',
 'Refren.',
 'Dobroć i łaska pójdą w ślad za mną *',
 'przez wszystkie dni życia',
 'i zamieszkam w domu Pana *',
 'po najdłuższe czasy.',
 'Refren.',
 'ŚPIEW PRZED EWANGELIĄ',
 'Por. Ez 33, 11',
 'Aklamacja: Chwała Tobie, Słowo Boże.',
 'Pan mówi: Nie chcę śmierci grzesznika,',
 'lecz pragnę, aby się nawrócił i miał życie.',
 'Aklamacj

In [ ]:
content_dic['PSALM RESPONSORYJNY'][18] = 'na oczach mych wrogów.'
content_dic['PSALM RESPONSORYJNY'].insert(19, 'Namaszczasz mi głowę olejkiem,')
content_dic['PSALM RESPONSORYJNY'].insert(21, 'Refren')
content_dic['PSALM RESPONSORYJNY'].insert(26, 'Refren')
content_dic['PSALM RESPONSORYJNY']

IndexError: list assignment index out of range

In [132]:
path = r"C:\Users\Acne\Pictures\Cyfrowa Biblia\automatyzacja\paper.gif"

image_size = 1080
with Image.open(path) as im:
    out = Image.new("RGB", (image_size, image_size), "white")
    x=0
    y=0
    size_x_left, size_y = im.size

    while x < image_size:
        while y < image_size:
            out.paste(im, (x, y))
            y += size_y
        y = 0
        x += size_x_left

# out.show()


In [133]:
tahoma = r"C:\Windows\Fonts\tahoma.ttf"
tahoma_bold = r"C:\Windows\Fonts\tahomabd.ttf"

In [134]:
current_path = WindowsPath(r'C:\Users\Acne\Pictures\Cyfrowa Biblia\\' + str(today))

os.makedirs(current_path, exist_ok=True)

In [91]:
def draw_text(out, name, current_path, size_x_left, size_y, font_size):

    font_size_small = int(0.75 * font_size)

    fnt = ImageFont.truetype(tahoma, font_size) 
    fnt_b = ImageFont.truetype(tahoma_bold, font_size)
    fnt_s = ImageFont.truetype(tahoma, font_size_small)

    size_x_left = 80
    size_x_right = image_size - 2 * size_x_left  
    size_y = int(1.2 * font_size)
    width = int(75 - 0.6 * font_size)

    out_func = copy.deepcopy(out)
    draw = ImageDraw.Draw(out_func) 

    draw.text((size_x_left, size_x_left), name, font=fnt_b, fill="red", anchor='lm')
    draw.text((size_x_right + size_x_left, size_x_left), content_dic[name][0], font=fnt_b, fill="red", anchor='rm') #TODO
    
    var_y = size_x_left + size_y
    draw.text((size_x_left, var_y), content_dic[name][1], font=fnt_s, fill="black", anchor='lm')
    
    lines = textwrap.wrap(content_dic[name][2], width=width)
    for line in lines:
        var_y += size_y
        draw.text((size_x_left, var_y), line, font=fnt_b, fill="black", anchor='lm')
    
    var_y += size_x_left
    
    for count, _ in enumerate(content_dic[name][3:-1]):
        lines = textwrap.wrap(content_dic[name][3+count], width=width, initial_indent='  ')
        lines[0] = lines[0].strip()
        # print('\n'.join(lines))
        space_length_regular = 15

        for line_count, line in enumerate(lines):
            x_word = size_x_left
            x_word_intended = 2 * size_x_left
            words = line.split(" ")
            words_length = sum(draw.textlength(w, font=fnt) for w in words)
            if len(words) == 1:
                words.append(' ')

            print(space_length_regular)
            if line_count == len(lines)-1:
                if words_length + (len(words)-1) * space_length_regular > size_x_right:
                    space_length_regular = (size_x_right - words_length) / (len(words) - 1)
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break
            
            space_length_intended = (size_x_right - x_word - words_length) / (len(words) - 1)
            space_length_regular = (size_x_right - words_length) / (len(words) - 1)
            
            if line_count == 0 and line != 'Bracia:' and 'Jezus powiedział' not in line:
                for word in words:
                    draw.text((x_word_intended, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word_intended += draw.textlength(word, font=fnt) + space_length_intended

            else:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
            var_y += size_y            
            
    draw.text((2*size_x_left, var_y+size_y), content_dic[name][-1], font=fnt_b, fill="black", anchor='lm')

    # out_func.show()

    returned = {'drawn_y': var_y+size_y, 'picture': out_func}

    return returned
    # out_func.save(current_path / f'{name}.png')

    out_func = copy.deepcopy(out)

# returned = draw_text(out, text, current_path, size_x_left, size_y, 44)
# returned['picture'].show()

In [ ]:
if "DRUGIE CZYTANIE" in content_dic:
    if 'ŚPIEW PRZED EWANGELIĄ' in content_dic["DRUGIE CZYTANIE"]:
        content_dic["DRUGIE CZYTANIE"] = content_dic["DRUGIE CZYTANIE"][:-6]

In [135]:
returned = draw_text(out, text, current_path, size_x_left, size_y, 4)
returned['picture'].show()

15
68.83333333333333
15
89.77777777777777
89.44444444444444
89.55555555555556
80.8
82.1
15
74.0
63.69230769230769
89.77777777777777
15
90.55555555555556
81.8
15
90.66666666666667
68.16666666666667
15


In [93]:
def draw_text_pagination_first(out, reading_list, current_path, size_x_left, size_y, font_size):

    font_size_small = int(0.75 * font_size)

    fnt = ImageFont.truetype(tahoma, font_size) 
    fnt_b = ImageFont.truetype(tahoma_bold, font_size)
    fnt_s = ImageFont.truetype(tahoma, font_size_small)

    size_x_left = int(25/12 * font_size)
    size_x_right = image_size - 2 * size_x_left
    size_y = int(1.2 * font_size)
    width = int(80 - 0.3 * size_x_left)

    out_func = copy.deepcopy(out)
    draw = ImageDraw.Draw(out_func) 

#TODO
    draw.text((size_x_left, size_x_left), reading_list[0], font=fnt_b, fill="red", anchor='lm')
    draw.text((size_x_right + size_x_left, size_x_left), reading_list[1], font=fnt_b, fill="red", anchor='rm') #TODO
    
    var_y = size_x_left + size_y
    draw.text((size_x_left, var_y), reading_list[2], font=fnt_s, fill="black", anchor='lm')
    
    lines = textwrap.wrap(reading_list[3], width=width)
    for line in lines:
        var_y += size_y
        draw.text((size_x_left, var_y), line, font=fnt_b, fill="black", anchor='lm')
    
    var_y += size_x_left
    
    for count, element in enumerate(reading_list[4:]):
        lines = textwrap.wrap(reading_list[4+count], width=width)
        space_length_regular = 15

        for line_count, line in enumerate(lines):
            x_word = size_x_left
            x_word_intended = 2 * size_x_left
            words = line.split(" ")
            words_length = sum(draw.textlength(w, font=fnt) for w in words)
            if len(words)==1:
                words.append(' ')

                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break

            if line_count == len(lines)-1:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break
            
            space_length_intended = (size_x_right - x_word - words_length) / (len(words) - 1)
            space_length_regular = (size_x_right - words_length) / (len(words) - 1)
           
            if line_count == 0 and line != 'Bracia:' and 'Jezus powiedział' not in line:
                for word in words:
                    draw.text((x_word_intended, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word_intended += draw.textlength(word, font=fnt) + space_length_intended

            else:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
            var_y += size_y            
            
    # out_func.show()

    returned = {'drawn_y': var_y+size_y, 'picture': out_func}

    return returned
    # out_func.save(current_path / f'{name}.png')

    out_func = copy.deepcopy(out)

In [94]:
def draw_text_pagination_second(out, reading_list, current_path, size_x_left, size_y, font_size):

    font_size_small = int(0.75 * font_size)
 
    fnt = ImageFont.truetype(tahoma, font_size) 
    fnt_b = ImageFont.truetype(tahoma_bold, font_size)
    fnt_s = ImageFont.truetype(tahoma, font_size_small)

    size_x_left = int(25/12 * font_size)
    size_x_right = image_size - 2 * size_x_left
    size_y = int(1.2 * font_size)
    width = int(80 - 0.3 * size_x_left)

    out_func = copy.deepcopy(out)
    draw = ImageDraw.Draw(out_func) 

    var_y = size_x_left
    # print(reading_list)
    lines = textwrap.wrap(reading_list[0], width=width)
    
    for count, element in enumerate(reading_list[0:-1]):
        lines = textwrap.wrap(reading_list[0+count], width=width)
        space_length_regular = 15

        for line_count, line in enumerate(lines):
            x_word = size_x_left
            x_word_intended = 2 * size_x_left
            words = line.split(" ")
            words_length = sum(draw.textlength(w, font=fnt) for w in words)
            if len(words)==1:
                words.append(' ')

                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break

            if line_count == len(lines)-1:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
                var_y += size_y  
                break
            
            space_length_intended = (size_x_right - x_word - words_length) / (len(words) - 1)
            space_length_regular = (size_x_right - words_length) / (len(words) - 1)
           
            if line_count == 0 and line != 'Bracia:' and 'Jezus powiedział' not in line:
                for word in words:
                    draw.text((x_word_intended, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word_intended += draw.textlength(word, font=fnt) + space_length_intended

            else:
                for word in words:
                    draw.text((x_word, var_y), word, font=fnt, fill="black", anchor='lm')
                    x_word += draw.textlength(word, font=fnt) + space_length_regular
            var_y += size_y            
            
    draw.text((2*size_x_left, var_y+size_y), reading_list[-1], font=fnt_b, fill="black", anchor='lm')

    # out_func.show()

    returned = {'drawn_y': var_y+size_y, 'picture': out_func}

    return returned
    # out_func.save(current_path / f'{name}.png')

    out_func = copy.deepcopy(out)

In [136]:
content_dic.keys()

dict_keys(['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA'])

In [137]:
min_font_before_pagination = 29
verse_break = 5
pagination_font_size = 34


for text in ['EWANGELIA']:
# .keys():
# ['PIERWSZE CZYTANIE']:
    if text != 'PSALM RESPONSORYJNY':
        font_size = 30
        returned = draw_text(out, text, current_path, size_x_left, size_y, font_size)

        # returned['picture'].show()
        # returned['picture'].save(current_path / f'{text}.png')

        while returned['drawn_y'] > image_size - 20:
            if font_size < min_font_before_pagination:
                pagination_dic = {}
                pagination_dic[f'{text} cz.1'] = content_dic[f'{text}'][:verse_break]
                pagination_dic[f'{text} cz.1'].insert(0, f'{text}' )
                pagination_dic[f'{text} cz.2'] = content_dic[f'{text}'][verse_break:]

                
                returned = draw_text_pagination_first(out, pagination_dic[f'{text} cz.1'], current_path, size_x_left, size_y, pagination_font_size)
                returned['picture'].show()
                returned['picture'].save(current_path / f'{text}1.png')

                returned = draw_text_pagination_second(out,  pagination_dic[f'{text} cz.2'], current_path, size_x_left, size_y, pagination_font_size)
                returned['picture'].show()
                returned['picture'].save(current_path / f'{text}2.png')

                break
            
            else:
                font_size -= 1
                returned = draw_text(out, text, current_path, size_x_left, size_y, font_size)
        
        print('font_size: ', font_size)
        
        

15
29.2
29.4
15
34.142857142857146
57.166666666666664
38.166666666666664
28.714285714285715
50.169270833333336
27.380859375
15
39.5
28.4
25.3
41.714285714285715
15
39.42857142857143
35.28794642857143
15
61.0
34.44618055555556
40.876953125
15
15
31.5
31.3
15
37.714285714285715
61.0
41.333333333333336
33.0
53.833333333333336
29.880859375
15
42.25
30.8
27.9
44.857142857142854
15
43.0
38.573660714285715
15
64.0
36.66840277777778
43.876953125
15
font_size:  29


In [ ]:
# returned = draw_text(out, 'PIERWSZE CZYTANIE', current_path, size_x_left, size_y, 26)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}.png')

In [138]:
font_size_psalm = 28
font_size_small_psalm = int(0.75 * font_size_psalm)

tahoma = r"C:\Windows\Fonts\tahoma.ttf"
tahoma_bold = r"C:\Windows\Fonts\tahomabd.ttf"

fnt_psalm = ImageFont.truetype(tahoma, font_size_psalm) 
fnt_b_psalm = ImageFont.truetype(tahoma_bold, font_size_psalm)
fnt_s_psalm = ImageFont.truetype(tahoma, font_size_small_psalm)

In [98]:
# content_dic["PSALM RESPONSORYJNY"].insert(16, 'Refren')
content_dic["PSALM RESPONSORYJNY"]

['J 8, 1-11',
 'W roku C, jeżeli Ewangelię J 8,1-11 odczytano w niedzielę, należy odczytać następującą: J 8,12-20 (druga z kolei)',
 'Chrystus ocala cudzołożnicę',
 'Słowa Ewangelii według Świętego Jana',
 'Jezus udał się na Górę Oliwną, ale o brzasku zjawił się znów w świątyni. Cały lud schodził się do Niego, a On, usiadłszy, nauczał ich.',
 'Wówczas uczeni w Piśmie i faryzeusze przyprowadzili do Niego kobietę, którą dopiero co pochwycono na cudzołóstwie, a postawiwszy ją pośrodku, powiedzieli do Niego: «Nauczycielu, tę kobietę dopiero co pochwycono na cudzołóstwie. W Prawie Mojżesz nakazał nam takie kamienować. A Ty co powiesz?» Mówili to, wystawiając Go na próbę,\xa0aby mieli o co Go oskarżyć.',
 'Lecz Jezus, schyliwszy się, pisał palcem po ziemi. A kiedy w dalszym ciągu Go pytali, podniósł się i rzekł do nich: «Kto z was jest bez grzechu, niech pierwszy rzuci w nią kamieniem». I powtórnie schyliwszy się, pisał na ziemi.',
 'Kiedy to usłyszeli, jeden po drugim zaczęli odchodzić, poc

In [99]:
print(len(content_dic["PSALM RESPONSORYJNY"][0]))
if len(content_dic["PSALM RESPONSORYJNY"][0]) > 35:
    content_dic["PSALM RESPONSORYJNY"][0] = content_dic["PSALM RESPONSORYJNY"][0].split(',')[0]
    
# del content_dic["PSALM RESPONSORYJNY"][-2]
# content_dic["PSALM RESPONSORYJNY"].insert(-1, 'pójdźcie, narody, oddajcie pokłon Panu,')
# content_dic["PSALM RESPONSORYJNY"].insert(-1, 'bo wielka światłość zstąpiła dzisiaj na ziemię.')
content_dic["PSALM RESPONSORYJNY"]

9


['J 8, 1-11',
 'W roku C, jeżeli Ewangelię J 8,1-11 odczytano w niedzielę, należy odczytać następującą: J 8,12-20 (druga z kolei)',
 'Chrystus ocala cudzołożnicę',
 'Słowa Ewangelii według Świętego Jana',
 'Jezus udał się na Górę Oliwną, ale o brzasku zjawił się znów w świątyni. Cały lud schodził się do Niego, a On, usiadłszy, nauczał ich.',
 'Wówczas uczeni w Piśmie i faryzeusze przyprowadzili do Niego kobietę, którą dopiero co pochwycono na cudzołóstwie, a postawiwszy ją pośrodku, powiedzieli do Niego: «Nauczycielu, tę kobietę dopiero co pochwycono na cudzołóstwie. W Prawie Mojżesz nakazał nam takie kamienować. A Ty co powiesz?» Mówili to, wystawiając Go na próbę,\xa0aby mieli o co Go oskarżyć.',
 'Lecz Jezus, schyliwszy się, pisał palcem po ziemi. A kiedy w dalszym ciągu Go pytali, podniósł się i rzekł do nich: «Kto z was jest bez grzechu, niech pierwszy rzuci w nią kamieniem». I powtórnie schyliwszy się, pisał na ziemi.',
 'Kiedy to usłyszeli, jeden po drugim zaczęli odchodzić, poc

In [139]:
out_psalm = copy.deepcopy(out)
draw = ImageDraw.Draw(out_psalm) 

y_distance = font_size_psalm * 0.8
y_further_distance = font_size_psalm * 1.1
x_distance = font_size_psalm * 2

draw.text((x_distance, y_further_distance*2), "PSALM RESPONSORYJNY", font=fnt_b_psalm, fill="red", anchor='lm')
draw.text((1000, y_further_distance*2), content_dic["PSALM RESPONSORYJNY"][0], font=fnt_b_psalm, fill="red", anchor='rm')
draw.text((x_distance, y_further_distance*3), "Refren: ", font=fnt_b_psalm, fill="red", anchor='lm')
draw.text((x_distance*3, y_further_distance*3), content_dic["PSALM RESPONSORYJNY"][1][7:], font=fnt_b_psalm, fill="black", anchor='lm')

# draw.text((100, 300), content_dic["PSALM RESPONSORYJNY"][3], font=fnt_psalm, fill="black", anchor='lm')
y_text = y_further_distance*4
for count, element in enumerate(content_dic["PSALM RESPONSORYJNY"][2:]):
    # lines = textwrap.wrap(second_element[2+count], width=80)
    # for line in lines:
    if "Refren" in element :
        y_text += y_further_distance / 2
        draw.text((x_distance, y_text), "Refren: ", font=fnt_b_psalm, fill="red", anchor='lm')
        draw.text((x_distance*3, y_text), content_dic["PSALM RESPONSORYJNY"][1][7:], font=fnt_b_psalm, fill="black", anchor='lm')
        y_text += y_further_distance  / 2
    elif "ŚPIEW PRZED EWANGELIĄ" in element:
        # print("NUMER: ", inner_count)
        acclamation = content_dic["PSALM RESPONSORYJNY"][count+2:]
        # if 'ŚPIEW' in acclamation[0]:
        #     acclamation.insert(1, '')
        print(acclamation)
        draw.text((x_distance, y_text), "AKLAMACJA PRZED EWANGELIĄ", font=fnt_b_psalm, fill="red", anchor='lm')
        draw.text((1000, y_text), acclamation[1], font=fnt_b_psalm, fill="red", anchor='rm')
        draw.text((x_distance, y_text+y_further_distance), "Aklamacja: ", font=fnt_b_psalm, fill="red", anchor='lm')
        draw.text((x_distance*4, y_text+y_further_distance), acclamation[2][10:], font=fnt_b_psalm, fill="black", anchor='lm')
        draw.text((x_distance*2, y_text+y_further_distance*2), acclamation[3] + ' ' + acclamation[4], font=fnt_psalm, fill="black", anchor='lm')
        # draw.text((x_distance*2, y_text+y_further_distance*2+y_distance), acclamation[4], font=fnt_psalm, fill="black", anchor='lm')
        draw.text((x_distance*2, y_text+y_further_distance*2+y_distance), acclamation[5], font=fnt_psalm, fill="black", anchor='lm')

        draw.text((x_distance, y_text+y_further_distance*3+y_distance), "Aklamacja: ", font=fnt_b_psalm, fill="red", anchor='lm') 
        draw.text((x_distance*4, y_text+y_further_distance*3+y_distance), acclamation[6][10:], font=fnt_b_psalm, fill="black", anchor='lm')

        break  

# TODO
    elif 'albo' in element:
        continue

    else:
        draw.text((x_distance*2, y_text), element, font=fnt_psalm, fill="black", anchor='lm')
    y_text += y_distance

out_psalm.show()
out_psalm.save(current_path / 'psalm.png')
# out_psalm.save('./2024-01-08/psalm.png')

out_psalm = copy.deepcopy(out)

['ŚPIEW PRZED EWANGELIĄ', 'Por. Ez 33, 11', 'Aklamacja: Chwała Tobie, Słowo Boże.', 'Pan mówi: Nie chcę śmierci grzesznika,', 'lecz pragnę, aby się nawrócił i miał życie.', 'Aklamacja: Chwała Tobie, Słowo Boże.']


IndexError: list index out of range

In [ ]:
# os.makedirs('./2024-01-08/', exist_ok=True)

In [ ]:
current_path

WindowsPath('C:/Users/Acne/Pictures/Cyfrowa Biblia/2024-01-26')

In [ ]:
# content_dic["EWANGELIA"]

In [140]:
content_dic.keys()

dict_keys(['PIERWSZE CZYTANIE', 'PSALM RESPONSORYJNY', 'EWANGELIA'])

In [ ]:
# content_dic['PIERWSZE CZYTANIE cz.1'] = content_dic['PIERWSZE CZYTANIE'][:6]
# content_dic['PIERWSZE CZYTANIE cz.2'] = content_dic['PIERWSZE CZYTANIE'][6:]
# returned = draw_text_pagination(out, 'PIERWSZE CZYTANIE cz.1', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}1.png')
# returned = draw_text_pagination_second(out, 'PIERWSZE CZYTANIE cz.2', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}2.png')

In [ ]:
# text = 'PIERWSZE CZYTANIE'
# content_dic[f'{text} cz.1'] = content_dic[f'{text}'][:6]
# content_dic[f'{text} cz.2'] = content_dic[f'{text}'][6:]
# returned = draw_text_pagination_first(out, f'{text} cz.1', current_path, size_x_left, size_y, 30)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}1.png')

# returned = draw_text_pagination_second(out, f'{text} cz.2', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}2.png')

In [ ]:
# content_dic['EWANGELIA'] = content_dic['EWANGELIA ']

In [ ]:
# text = 'EWANGELIA'
# content_dic['EWANGELIA cz.1'] = content_dic['EWANGELIA'][:7]
# content_dic['EWANGELIA cz.2'] = content_dic['EWANGELIA'][7:]
# returned = draw_text_pagination_first(out, 'EWANGELIA cz.1', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}1.png')

# returned = draw_text_pagination_second(out, 'EWANGELIA cz.2', current_path, size_x_left, size_y, 32)
# returned['picture'].show()
# returned['picture'].save(current_path / f'{text}2.png')

In [ ]:
content_dic['PIERWSZE CZYTANIE']

['2 Tm 1,1-8',
 'Żywa wiara Tymoteusza',
 'Czytanie z Drugiego listu świętego Pawła Apostoła do Tymoteusza',
 'Paweł, z woli Boga apostoł Chrystusa Jezusa, posłany dla głoszenia życia obiecanego w Chrystusie Jezusie, do Tymoteusza, swego umiłowanego dziecka. Łaska, miłosierdzie, pokój od Boga Ojca i Chrystusa Jezusa, naszego Pana.',
 'Dziękuję Bogu, któremu służę jak moi przodkowie z czystym sumieniem, gdy zachowuję nieprzerwaną pamięć o tobie w moich modlitwach. W nocy i we dnie pragnę cię zobaczyć, pomny na twoje łzy, by napełniła mnie radość na wspomnienie wiary bez obłudy, jaka jest w tobie; ona to zamieszkała pierwej w twej babce Lois i w twej matce Eunice, a pewien jestem, że mieszka i w tobie.',
 'Z tej właśnie przyczyny przypominam ci, abyś rozpalił na nowo charyzmat Boży, który jest w tobie przez włożenie moich rąk. Albowiem nie dał nam Bóg ducha bojaźni, ale mocy i miłości, i trzeźwego myślenia. Nie wstydź się zatem świadectwa naszego Pana ani mnie, Jego więźnia, lecz weź udz

In [ ]:
content_dic['EWANGELIA']

['Łk 10,1-9',
 'Rozesłanie uczniów',
 'Słowa Ewangelii według świętego Łukasza',
 'Spośród swoich uczniów wyznaczył Pan jeszcze innych, siedemdziesięciu dwóch, i wysłał ich po dwóch przed sobą do każdego miasta i miejscowości, dokąd sam przyjść zamierzał.',
 'Powiedział też do nich: „Żniwo wprawdzie wielkie, ale robotników mało; proście więc Pana żniwa, żeby wyprawił robotników na swoje żniwo. Idźcie, oto was posyłam jak owce między wilki. Nie noście ze sobą trzosa ani torby, ani sandałów; i nikogo w drodze nie pozdrawiajcie.',
 'Gdy do jakiego domu wejdziecie, najpierw mówcie: «Pokój temu domowi!» Jeśli tam mieszka człowiek godny pokoju, wasz pokój spocznie na nim; jeśli nie, powróci do was.',
 'W tym samym domu zostańcie, jedząc i pijąc, co mają: bo zasługuje robotnik na swą zapłatę.',
 'Nie przechodźcie z domu do domu. Jeśli do jakiego miasta wejdziecie i przyjmą was, jedzcie, co wam podadzą; uzdrawiajcie chorych, którzy tam są, i mówcie im: «Przybliżyło się do was królestwo Boże»”.

In [ ]:
URL = f"https://www.vaticannews.va/en/word-of-the-day/{str(today).replace('-', '/')}.html"

page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
content_list = soup.find_all("div", "section__content")[:2]
content_list_text = [x.get_text().split('\n') for x in content_list]
for num, y in enumerate(content_list_text):
    content_list_text[num] = [x.replace("\"","") for x in y]

content_list_text

[['',
  'A reading from the Second Letter of St. Paul to Timothy 2 Tm 1:1-8',
  'Paul, an Apostle of Christ Jesus by the will of God for the promise of life in Christ Jesus, to Timothy, my dear child: grace, mercy, and peace from God the Father and Christ Jesus our Lord.',
  'I am grateful to God, whom I worship with a clear conscience as my ancestors did, as I remember you constantly in my prayers, night and day. I yearn to see you again, recalling your tears, so that I may be filled with joy, as I recall your sincere faith that first lived in your grandmother Lois and in your mother Eunice and that I am confident lives also in you.',
  'For this reason, I remind you to stir into flame the gift of God that you have through the imposition of my hands. For God did not give us a spirit of cowardice but rather of power and love and self-control. So do not be ashamed of your testimony to our Lord, nor of me, a prisoner for his sake; but bear your share of hardship for the Gospel with the s

In [163]:
str(today)

'2024-03-21'

In [164]:
from instagrapi import Client
cl = Client()
cl.login('cyfrowa_biblia', 'Etgohome29.')

True

In [165]:
current_path_folder = WindowsPath(r"C:\Users\Acne\Pictures\Cyfrowa Biblia\web\\" + str(today))


In [166]:
general_order = ['mateusz', 'PIERWSZE', 'PSALM', 'DRUGIE', 'EWANGELIA']
current_order = []
# for file in os.listdir(f'./web/{today}/'):
current_order = [r'C:\Users\Acne\Pictures\Cyfrowa Biblia\automatyzacja\Cyber_bible\web\\'+ str(today) + r'\\' + x for y in general_order for x in os.listdir(f'./web/{today}/') if y in x]
current_order

['C:\\Users\\Acne\\Pictures\\Cyfrowa Biblia\\automatyzacja\\Cyber_bible\\web\\\\2024-03-21\\\\mateusztront_My_covenant_with_you_is_this_you_are_to_become_the_f0a7f4db-f6ef-406e-9a4a-9e1f673e07e2.jpg',
 'C:\\Users\\Acne\\Pictures\\Cyfrowa Biblia\\automatyzacja\\Cyber_bible\\web\\\\2024-03-21\\\\PIERWSZE CZYTANIE.jpg',
 'C:\\Users\\Acne\\Pictures\\Cyfrowa Biblia\\automatyzacja\\Cyber_bible\\web\\\\2024-03-21\\\\PSALM RESPONSORYJNY.jpg',
 'C:\\Users\\Acne\\Pictures\\Cyfrowa Biblia\\automatyzacja\\Cyber_bible\\web\\\\2024-03-21\\\\EWANGELIA.jpg']

In [167]:
cap = """ Abram padł na oblicze, a Bóg tak do niego mówił:
«Oto moje przymierze z tobą: staniesz się ojcem mnóstwa narodów. Nie będziesz więc odtąd nazywał się Abram, lecz imię twoje będzie Abraham, bo uczynię ciebie ojcem mnóstwa narodów. Sprawię, że będziesz niezmiernie płodny, tak że staniesz się ojcem narodów i pochodzić będą od ciebie królowie.
Przymierze moje, które zawieram pomiędzy Mną a tobą oraz twoim potomstwem, będzie trwało przez pokolenia jako przymierze wieczne, abym był Bogiem twoim, a potem twego potomstwa. I oddaję tobie i twym przyszłym potomkom kraj, w którym przebywasz, cały kraj Kanaan, jako własność na wieki i będę ich Bogiem».
Potem rzekł Bóg do Abrahama: «Ty zaś, a po tobie twoje potomstwo przez wszystkie pokolenia zachowujcie przymierze ze Mną».

***
Grafika wykonana za pomocą sztucznej inteligencji.

#Bible #Biblia #czytanianadzis #słowoboze #bing #dalle3 #midjourney #catholicchurch #religion #religia #HolySpirit #DuchŚwięty #Zmartwychwstanie #Resurrection
"""

In [168]:
cl.album_upload(paths=current_order, caption=cap)

Media(pk=3328870424509799006, id='3328870424509799006_60024442982', code='C4yhgOUNdpe', taken_at=datetime.datetime(2024, 3, 21, 20, 18, 20, tzinfo=TzInfo(UTC)), media_type=8, image_versions2={'candidates': [{'width': 1024, 'height': 1024, 'url': 'https://scontent-waw2-2.cdninstagram.com/v/t51.29350-15/434112943_7186081351440592_6901392831191148104_n.jpg?se=7&stp=dst-jpg_e35&_nc_ht=scontent-waw2-2.cdninstagram.com&_nc_cat=104&_nc_ohc=Bt8ZfRRBAZ4AX8yc4lA&edm=AFrf_vIBAAAA&ccb=7-5&ig_cache_key=MzMyODg3MDQyMDYxNzU1ODQwMw%3D%3D.2-ccb7-5&oh=00_AfCKox629jfnu4LqxAoeRYyDBslEk3x2yxV9MJjFJjAdhQ&oe=6600CC46&_nc_sid=d31fa1', 'scans_profile': ''}, {'width': 320, 'height': 320, 'url': 'https://scontent-waw2-2.cdninstagram.com/v/t51.29350-15/434112943_7186081351440592_6901392831191148104_n.jpg?stp=dst-jpg_e35_s320x320&_nc_ht=scontent-waw2-2.cdninstagram.com&_nc_cat=104&_nc_ohc=Bt8ZfRRBAZ4AX8yc4lA&edm=AFrf_vIBAAAA&ccb=7-5&ig_cache_key=MzMyODg3MDQyMDYxNzU1ODQwMw%3D%3D.2-ccb7-5&oh=00_AfCrWe5ZY1_Qb-IJiEgQw

In [ ]:
cl.photo_upload(WindowsPath('C:/Users/Acne/Pictures/Cyfrowa Biblia/automatyzacja/Cyber_bible/web/2024-03-06/mateusztront_Therefore_whoever_breaks_one_of_the_least_of_these_970a3d77-de79-4f26-93ee-2d5d5f7f02fc.png'), 'Description')

Media(pk=3317921935806318197, id='3317921935806318197_60024442982', code='C4LoGvoNkZ1', taken_at=datetime.datetime(2024, 3, 6, 17, 45, 38, tzinfo=TzInfo(UTC)), media_type=1, image_versions2={'candidates': [{'width': 1024, 'height': 1024, 'url': 'https://scontent-waw1-1.cdninstagram.com/v/t51.29350-15/431714566_1628923351181954_1314764076689941582_n.jpg?se=7&stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDI0eDEwMjQuc2RyIn0&_nc_ht=scontent-waw1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=halN0V307GUAX-WHXJ4&edm=ACqnv0EBAAAA&ccb=7-5&ig_cache_key=MzMxNzkyMTkzNTgwNjMxODE5Nw%3D%3D.2-ccb7-5&oh=00_AfCfmi_Xjayj6teOxwkuocxN6KPmIc1JU4_DI019WDiGKQ&oe=65EC4659&_nc_sid=7698e5', 'scans_profile': 'e35', 'estimated_scans_sizes': [16059, 32118, 48177, 64236, 80295, 96916, 117325, 130661, 144531]}, {'width': 360, 'height': 360, 'url': 'https://scontent-waw1-1.cdninstagram.com/v/t51.29350-15/431714566_1628923351181954_1314764076689941582_n.jpg?stp=dst-jpg_e35_s360x360&efg=eyJ2ZW5jb2RlX3RhZyI6Imlt